In [ ]:
!pip install pyrankvote

In [1]:
import pyrankvote
import ast
import pandas as pd
from pyrankvote import Candidate, Ballot


def rename_index_col_to_ballot_id(df):
    df.rename(columns={'Unnamed: 0': 'ballot_id'}, inplace=True)
    return df

def get_cand_list(df):
    list_of_unique_vals_in_dataframe = df.iloc[:, 1].value_counts().index
    cand_list = list(filter(lambda x: x != '0', list_of_unique_vals_in_dataframe))
    return cand_list

def initialize_cand_objs(cand_list):
    if isinstance(cand_list, str):
        return [Candidate(c) for c in cand_list.split(', ')]
    else:
        return [Candidate(c) for c in cand_list]

# def initialize_cand_objs_in_df_cols(df):
#     cands_df = df.iloc[:, 1:]  # getting all columns with candidate names in them
#     cands_df = cands_df.applymap(lambda x: create_cand(x))  # making them into pyrankvote candidate objects
#     cands_df['ballot_id'] = df.iloc[:, 0]  # stitching df back with ballot_ids from old dataframe
#     return cands_df

def get_cands_into_single_cell(df):
    df['candidate_list'] = df.iloc[:, 1:].agg(", ".join, axis=1)
    # cands_to_put_in_single_cell = df.iloc[:, 1:].columns.to_list()
    # df['candidate_list'] = str(cands_to_put_in_single_cell)
    # df['candidate_list'] = df['candidate_list'].apply(lambda x: ast.literal_eval(x))
    return df

def initialize_ballot_objs(df):
    ballot_objects = []
    for index, value in enumerate(df['candidate_list']):
        ballot = Ballot(ranked_candidates=value)
        ballot_objects.append(ballot)
    return ballot_objects

def run_election(list_of_cand_objs, election_df):
    return pyrankvote.instant_runoff_voting(list_of_cand_objs, election_df['ballots'], pick_random_if_blank=True)

def rm_invalid_rows(df):
    return df[df['candidate_list'] != '0']

def run_all_steps(df):
    election_dict = {}
    #df = pd.read_csv('/Users/shvarugh/Documents/myGit/ranked-choice-voting/master_elections.csv')
#     df = rename_index_col_to_ballot_id(df)
    df = df.dropna(axis=1, how='all')
    df = df.drop(['ballot_id','filename'], axis=1)
#     print(df.shape)
    df = get_cands_into_single_cell(df)

    df['candidate_list'] = df['candidate_list'].apply(lambda x: x.replace('0, ', '').replace(', 0', ''))
    df = rm_invalid_rows(df)
    df['candidate_list'] = df['candidate_list'].apply(lambda x: initialize_cand_objs(x))

    ballots = initialize_ballot_objs(df)
    df['ballots'] = ballots
    
    cand_list = get_cand_list(df)
    cand_list = initialize_cand_objs(cand_list)
#     print("cand_list: ",cand_list)
#     print(df.head())
    election = run_election(cand_list, df)
    winner = election.get_winners()
    print("get_winners",winner)
    election_dict['winner'] = winner
    #df["irv_winner"] = winner
    print("rounds",len(election.rounds))
    election_dict['rounds'] = len(election.rounds)

#     open file and write results. read winners votes from file and write to df
    #print("register_round_results",election.register_round_results(election.rounds[-1]))

    print("election:", election)
    election_dict['election'] = election

#     print("election type:", type(election))
    election_dict['register']=election.register_round_results(election.rounds[-1])

    print('hi')
    return df, election_dict

def read_votes(file, election_dict):
    with open(file, 'r') as file1:
        count = 0
        while True: 
            count += 1

            # Get next line from file 
            line = file1.readline()
            if not line: 
                break
            strip_line = line.strip()
    #         print('*',strip_line,'*')
            if 'ROUND' in strip_line or 'FINAL' in strip_line:
                start_round = True
                if 'ROUND 1' in strip_line or 'FINAL' in strip_line:
                    monitor_round=True
                    print('*',strip_line,'*')
                else:
                    monitor_round=False
            if 'ROUND' not in strip_line and  'FINAL' not in strip_line:
                start_round = False
                if winner in line and monitor_round:
                    print('*',strip_line,'*')
                    if 'firstRoundVotes' not in election_dict:
                        election_dict['firstRoundVotes']=[int(s) for s in strip_line.split() if s.isdigit()].pop()
                    else:
                        election_dict['finalRoundVotes']=[int(s) for s in strip_line.split() if s.isdigit()].pop()
                    
    file1.close() 
    






In [3]:
df = pd.read_csv('/Users/shvarugh/Documents/myGit/ranked-choice-voting/master_elections.csv')
df = rename_index_col_to_ballot_id(df)
# df['irv_winner']=""

/Users/shvarugh/.pyenv/versions/jupyter3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (5,6) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
pd.set_option('max_colwidth', 8000)
pd.options.display.max_columns = None
pd.options.display.float_format = '{:.4f}'.format
pd.options.display.max_rows = 10000

In [ ]:
#df[df['filename']=="election_05-04-2020_08-40-23_5cands_16noise.csv"]

In [5]:
election_ids= df['filename'].unique()
#election_ids

In [6]:
all_elections_dict = {}

In [7]:
df.head()

,ballot_id,candidate_1,candidate_2,candidate_3,candidate_4,candidate_5,candidate_6,filename
0,0,candidate_1,candidate_2,candidate_3,candidate_4,candidate_5,NaN,election_05-04-2020_08-40-23_5cands_16noise.csv
1,1,candidate_1,candidate_2,candidate_3,candidate_4,candidate_5,NaN,election_05-04-2020_08-40-23_5cands_16noise.csv
2,2,candidate_1,candidate_2,candidate_3,candidate_4,candidate_5,NaN,election_05-04-2020_08-40-23_5cands_16noise.csv
3,3,candidate_1,candidate_2,candidate_3,0,0,NaN,election_05-04-2020_08-40-23_5cands_16noise.csv
4,4,candidate_1,candidate_2,candidate_3,candidate_4,candidate_5,NaN,election_05-04-2020_08-40-23_5cands_16noise.csv


In [10]:
for election_num, election in enumerate(election_ids):
#     election_dict[election]=
    #print(election_num, election)
    if election_num < 2:
        print("***********Election: ", election_num)
        df_election = df.loc[df['filename']==election]
        #print(df_election.shape)
        
        temp_df,election_dict = run_all_steps(df_election)
        #print(election_dict)
        if len(election_dict['winner'])==1:
            winner = election_dict['winner'][0].name
            election_dict['winner'] = winner
        else:
            print(election, "has more than one winner")
        result_file = 'out.txt'
        with open(result_file, 'w') as file1:
            print(election_dict['election'], file=file1)
        read_votes(result_file, election_dict)
        all_elections_dict[election_num] = election_dict


***********Election:  0
get_winners [<Candidate('candidate_1')>]
rounds 4
election: ROUND 1
Candidate      Votes  Status
-----------  -------  --------
candidate_1     9129  Hopeful
candidate_5     9010  Hopeful
candidate_2     8455  Hopeful
candidate_3     7911  Hopeful
candidate_4     6858  Rejected

ROUND 2
Candidate      Votes  Status
-----------  -------  --------
candidate_1    10902  Hopeful
candidate_5    10641  Hopeful
candidate_3    10078  Hopeful
candidate_2     9742  Rejected
candidate_4        0  Rejected

ROUND 3
Candidate      Votes  Status
-----------  -------  --------
candidate_1    14068  Hopeful
candidate_3    13675  Hopeful
candidate_5    13620  Rejected
candidate_2        0  Rejected
candidate_4        0  Rejected

FINAL RESULT
Candidate      Votes  Status
-----------  -------  --------
candidate_1    20719  Elected
candidate_3    20644  Rejected
candidate_5        0  Rejected
candidate_2        0  Rejected
candidate_4        0  Rejected

hi
* ROUND 1 *
* candidat

In [11]:
all_elections_dict

{0: {'winner': 'candidate_1',
  'rounds': 4,
  'election': <ElectionResults(5 rounds)>,
  'register': None,
  'firstRoundVotes': 9129,
  'finalRoundVotes': 20719},
 1: {'winner': 'candidate_1',
  'rounds': 3,
  'election': <ElectionResults(4 rounds)>,
  'register': None,
  'firstRoundVotes': 13918,
  'finalRoundVotes': 22945}}